In [21]:

pip install --upgrade linkedin-api


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
pip install facebook-sdk


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
import openpyxl
from datetime import datetime
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from linkedin_api import Linkedin
import facebook

# Function to upload videos/posts to YouTube
def upload_to_youtube(title, description, file_path, tags, thumbnail, privacy, comments, publish_time):
    # Set up YouTube API client
    credentials = Credentials.from_authorized_user_file('credentials.json', ['https://www.googleapis.com/auth/youtube.force-ssl'])
    youtube = build('youtube', 'v3', credentials=credentials)
    
    # Create the video metadata
    request_body = {
        'snippet': {
            'title': title,
            'description': description,
            'tags': tags.split(','),
        },
        'status': {
            'privacyStatus': privacy,
            'selfDeclaredMadeForKids': False,
            'commentModerationSetting': 'moderate' if comments.lower() == 'yes' else 'none',
        },
    }
    
    # Upload the video
    insert_request = youtube.videos().insert(
        part='snippet,status',
        body=request_body,
        media_body=MediaFileUpload(file_path),
        notifySubscribers=False
    )
    response = insert_request.execute()
    
    # Set the thumbnail for the video
    if thumbnail:
        youtube.thumbnails().set(
            videoId=response['id'],
            media_body=MediaFileUpload(thumbnail)
        ).execute()
    
    # Set the publish time for the video
    youtube.videos().update(
        part='snippet',
        body={
            'id': response['id'],
            'snippet': {
                'scheduledStartTime': publish_time.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
            }
        }
    ).execute()

# Function to upload posts to LinkedIn
def upload_to_linkedin(title, description, file_path, tags, privacy, publish_time):
    # Set up LinkedIn API client
    client = Linkedin(access_token='your-access-token')
    
    # Create the post metadata
    post_data = {
        'author': 'urn:li:organization:your-organization-id',
        'lifecycleState': 'PUBLISHED',
        'specificContent': {
            'com.linkedin.ugc.ShareContent': {
                'shareCommentary': {
                    'text': description,
                },
                'media': [
                    {
                        'status': 'READY',
                        'media': file_path,
                        'title': {
                            'text': title
                        }
                    }
                ],
                'shareMediaCategory': 'VIDEO',
            }
        },
        'visibility': {
            'com.linkedin.ugc.MemberNetworkVisibility': privacy.title()
        },
    }
    
    # Upload the post
    client._post('/media/upload', post_data)
    
    # Publish the post at the specified time
    client._post(f'/shares?owner=urn:li:organization:your-organization-id&sharesEnabled=true&visibility={privacy.title()}&scheduledStartTime={publish_time.timestamp()*1000}', post_data)

# Function to upload posts to Facebook
def upload_to_facebook(title, description, file_path, tags, privacy, publish_time):
    # Set up Facebook API client
    access_token = 'your-access-token'
    graph = facebook.GraphAPI(access_token)
    
    # Create the post metadata
    post_data = {
        'title': title,
        'description': description,
        'source': open(file_path, 'rb'),
    }
    
    # Upload the post
    graph.put_object('me', 'videos', **post_data)
    
    # Set the privacy for the post
    graph.put_object('me', 'permissions', privacy={'value': privacy})
    
    # Publish the post at the specified time
    graph.put_object('me', 'feed', message=title, scheduled_publish_time=int(publish_time.timestamp()))
    
# Load the Excel sheet
workbook = openpyxl.load_workbook('/Book1.xlsx')
worksheet = workbook.active

# Iterate over each row in the Excel sheet
for row in worksheet.iter_rows(min_row=2, values_only=True):
    title, description, file_path, platform, publish_date, tags, thumbnail, privacy, comments = row
    
    # Convert the publish_date string to a datetime object
    publish_time = datetime.strptime(publish_date, '%Y-%m-%d %H:%M:%S')
    
    # Upload the video/post to the specified platform
    if platform.lower() == 'youtube':
        upload_to_youtube(title, description, file_path, tags, thumbnail, privacy, comments, publish_time)
    elif platform.lower() == 'linkedin':
        upload_to_linkedin(title, description, file_path, tags, privacy, publish_time)
    elif platform.lower() == 'facebook':
        upload_to_facebook(title, description, file_path, tags, privacy, publish_time)



ValueError: ignored